In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

import torch
from detectron2.config import get_cfg
from detectron2.engine.defaults import DefaultPredictor
from detectron2.data.datasets import register_coco_instances
from detectron2.data import get_detection_dataset_dicts
from detectron2.data.detection_utils import read_image
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode

import numpy as np
import os
import json
import cv2
from pycocotools import mask

import json
from detectron2.data.detection_utils import read_image
from PIL import Image
import numpy

import cv2
from matplotlib import pyplot as plt
from detectron2.utils.visualizer import Visualizer, VisImage

from detectron2.config import configurable
from detectron2.modeling.roi_heads.box_head import build_box_head
from detectron2.modeling.roi_heads.roi_heads import StandardROIHeads

from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, fast_rcnn_inference
from detectron2.structures import Boxes, ImageList, Instances, pairwise_iou
from detectron2.layers import ShapeSpec, batched_nms
from typing import Dict, List, Optional, Tuple
from detectron2.modeling.poolers import ROIPooler
from torch.nn import functional as F

from detectron2.layers import ShapeSpec, batched_nms, cat, cross_entropy, nonzero_tuple
from detectron2.utils.events import get_event_storage
from detectron2.evaluation.coco_evaluation import COCOEvaluator

from detectron2.engine.defaults import DefaultTrainer
import logging
from detectron2.modeling import build_model
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.layers import ShapeSpec
import torch

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

class NonCocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name)

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128


In [2]:
training_categ = ['chess', 'skiing', 'weightlifting', 'climbing', 'cricket', 'flying', 'hockey', 'soccer', 'volleyball', 'tennis', 'skateboarding', 'swimming', 'rowing', 'roller skating', 'horse racing', 'steeplechase', 'jogging', 'gymnastics', 'golf', 'diving', 'car racing', 'boxing', 'bowling', 'billiard', 'beach volleyball', 'basketball', 'baseball', 'jumping', 'running', 'acrobatics', 'airplane', 'glider', 'helicopter', 'hot-air_balloon', 'bicycle', 'camper', 'convertible', 'jeep', 'limousine', 'sedan', 'taxi', 'wagon', 'carriage', 'motorcycle', 'bus', 'minibus', 'tram', 'trolleybus', 'road sign', 'traffic police', 'zebra crossing', 'boat', 'ferry', 'gondola', 'motorboat', 'sailing vessel', 'ship', 'yacht', 'sleigh', 'rocket', 'spaceship', 'train', 'car transporter', 'dumper', 'garbage truck', 'lorry', 'pickup', 'tow truck', 'truck', 'van', 'bulldozer', 'digger', 'forklift', 'tractor', 'artist', 'sculptor', 'accordionist', 'piper', 'cellist', 'clarinetist', 'conductor', 'flute player', 'guitar player', 'opera singer', 'percussionist', 'piano player', 'rapper', 'saxophonist', 'singer', 'trombonist', 'trumpeter', 'violinist', 'ballet dancer', 'cameraman', 'clown', 'dancer', 'makeup artist', 'photographer', 'writer', 'figure skating', 'off road motorcycling', 'motorcycle racing', 'baby carriage', 'fire engine', 'fireman', 'police car', 'police helicopter', 'mounted police', 'policeman', 'wheelchair', 'fishing', 'hunting', 'tank', 'hang gliding', 'rhythmic gymnastics', 'horse sleigh', 'ambulance', 'dog sleigh', 'military helicopter', 'police boat', 'motorcycle police', 'soldier', 'double-decker', 'bicycle racing', 'handball', 'armoured personnel carrier', 'military truck', 'rickshaw', 'scooter', 'pole vaulting']

cfg = get_cfg()
cfg.merge_from_file('/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml')
cfg.MODEL.WEIGHTS = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x/137260431/model_final_a54504.pkl'

cfg.DATALOADER.NUM_WORKERS = 1
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 0
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1400)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.TEST.EVAL_PERIOD = 2000
cfg.MODEL.DEVICE='cuda:1'

for folder_name in training_categ:
    folder_name = folder_name.replace(' ','_')
    try:
        DatasetCatalog.remove(folder_name)
        MetadataCatalog.remove(folder_name)
        print('Update dataset')
    except:
        print('New dataset')
    #register_coco_instances(folder_name,{},'/host/comparison/'+folder_name+'/'+folder_name+'_train.json','/host/comparison/'+folder_name+'/data/')
    register_coco_instances(folder_name,{},'/host/mic21-framework/server/uploads/'+folder_name+'_gt.json','/host/mic21-framework/server/uploads/'+folder_name+'/')
    
    try:
        del trainer
    except:
        print("Starting...")
    
    ind = 0
    img = DatasetCatalog.get(folder_name)[ind]['file_name']
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #v = Visualizer(img,MetadataCatalog.get(folder_name))
    #v.draw_dataset_dict(DatasetCatalog.get(folder_name)[ind])
    print(folder_name)

    #plt.figure(figsize=(15, 15))
    #plt.imshow(v.get_output().get_image())
    #plt.show()
    
    cfg.DATASETS.TRAIN = (folder_name,)
    cfg.DATASETS.TEST = ()
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get(folder_name).thing_classes)
    
    torch.cuda.init()
        
    trainer = NonCocoTrainer(cfg)
    trainer.resume_or_load(resume=False)
    try:
        trainer.train()
    except:
        continue
    trainer.checkpointer.save(folder_name)


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



New dataset
Starting...
guitar_player
[03/04 11:14:09 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, e

WARNING [03/04 11:14:09 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/04 11:14:09 d2.data.datasets.coco]: Loaded 127 images in COCO format from /host/mic21-framework/server/uploads/guitar_player_gt.json
[03/04 11:14:09 d2.data.build]: Removed 0 images with no usable annotations. 127 images left.
[03/04 11:14:09 d2.data.build]: Distribution of instances among all 5 categories:
|   category    | #instances   |   category    | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
| guitar player | 124          |  microphone   | 54           |   guitar   | 80           |
| digital gui.. | 65           | microphone .. | 52           |            |              |
|     total     | 375          |               |              |            |              |
[03/04 11:14:09 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: 

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (6, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (6,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (20, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (20,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (5, 256, 1, 1) 

[03/04 11:14:10 d2.engine.train_loop]: Starting training from iteration 0


...

WARNING [03/05 04:23:09 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/05 04:23:09 d2.data.datasets.coco]: Loaded 139 images in COCO format from /host/mic21-framework/server/uploads/pole_vaulting_gt.json
[03/05 04:23:09 d2.data.build]: Removed 0 images with no usable annotations. 139 images left.
[03/05 04:23:09 d2.data.build]: Distribution of instances among all 9 categories:
|   category   | #instances   |   category   | #instances   |   category    | #instances   |
|:------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
|    briefs    | 60           |    shorts    | 74           | athletic sock | 107          |
| pole vaulter | 141          | jumping pole | 137          |   standard    | 66           |
|   crossbar   | 95           |   tank top   | 125          |  jumper shoe  | 248          |
|              |              |              |              |               |       

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (10, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (10,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (36, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (36,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (9, 256, 1, 1

[03/05 04:23:09 d2.engine.train_loop]: Starting training from iteration 0
[03/05 04:23:29 d2.utils.events]:  eta: 0:24:05  iter: 19  total_loss: 3.044  loss_cls: 1.094  loss_box_reg: 0.9486  loss_mask: 0.6168  loss_rpn_cls: 0.09174  loss_rpn_loc: 0.2865  time: 0.9850  data_time: 0.3459  lr: 0.001  max_mem: 0M
[03/05 04:23:49 d2.utils.events]:  eta: 0:24:14  iter: 39  total_loss: 2.519  loss_cls: 0.8204  loss_box_reg: 0.8441  loss_mask: 0.4553  loss_rpn_cls: 0.07566  loss_rpn_loc: 0.2682  time: 0.9817  data_time: 0.3251  lr: 0.001  max_mem: 0M
[03/05 04:24:08 d2.utils.events]:  eta: 0:24:34  iter: 59  total_loss: 2.208  loss_cls: 0.6931  loss_box_reg: 0.8788  loss_mask: 0.3564  loss_rpn_cls: 0.04825  loss_rpn_loc: 0.2632  time: 0.9789  data_time: 0.3266  lr: 0.001  max_mem: 0M
[03/05 04:24:27 d2.utils.events]:  eta: 0:23:23  iter: 79  total_loss: 2.027  loss_cls: 0.651  loss_box_reg: 0.7658  loss_mask: 0.3183  loss_rpn_cls: 0.04875  loss_rpn_loc: 0.2449  time: 0.9708  data_time: 0.3053 

[03/05 04:34:01 d2.utils.events]:  eta: 0:13:10  iter: 679  total_loss: 0.9421  loss_cls: 0.2058  loss_box_reg: 0.3888  loss_mask: 0.1573  loss_rpn_cls: 0.01214  loss_rpn_loc: 0.171  time: 0.9573  data_time: 0.2881  lr: 0.001  max_mem: 0M
[03/05 04:34:20 d2.utils.events]:  eta: 0:12:53  iter: 699  total_loss: 0.9458  loss_cls: 0.2363  loss_box_reg: 0.3763  loss_mask: 0.1544  loss_rpn_cls: 0.01447  loss_rpn_loc: 0.1792  time: 0.9566  data_time: 0.2975  lr: 0.001  max_mem: 0M
[03/05 04:34:39 d2.utils.events]:  eta: 0:12:34  iter: 719  total_loss: 1.013  loss_cls: 0.2363  loss_box_reg: 0.3954  loss_mask: 0.1494  loss_rpn_cls: 0.013  loss_rpn_loc: 0.1473  time: 0.9575  data_time: 0.3217  lr: 0.001  max_mem: 0M
[03/05 04:34:58 d2.utils.events]:  eta: 0:12:15  iter: 739  total_loss: 0.9285  loss_cls: 0.1907  loss_box_reg: 0.3635  loss_mask: 0.1442  loss_rpn_cls: 0.01437  loss_rpn_loc: 0.1864  time: 0.9571  data_time: 0.3046  lr: 0.001  max_mem: 0M
[03/05 04:35:17 d2.utils.events]:  eta: 0:11

[03/05 04:44:30 d2.utils.events]:  eta: 0:02:34  iter: 1339  total_loss: 0.825  loss_cls: 0.1642  loss_box_reg: 0.3099  loss_mask: 0.1406  loss_rpn_cls: 0.009872  loss_rpn_loc: 0.1563  time: 0.9552  data_time: 0.3141  lr: 5e-05  max_mem: 0M
[03/05 04:44:50 d2.utils.events]:  eta: 0:02:14  iter: 1359  total_loss: 0.7072  loss_cls: 0.1434  loss_box_reg: 0.2843  loss_mask: 0.1291  loss_rpn_cls: 0.01001  loss_rpn_loc: 0.1482  time: 0.9556  data_time: 0.3376  lr: 5e-05  max_mem: 0M
[03/05 04:45:10 d2.utils.events]:  eta: 0:01:55  iter: 1379  total_loss: 0.7944  loss_cls: 0.1543  loss_box_reg: 0.3008  loss_mask: 0.1451  loss_rpn_cls: 0.01031  loss_rpn_loc: 0.1485  time: 0.9559  data_time: 0.3313  lr: 5e-05  max_mem: 0M
[03/05 04:45:29 d2.utils.events]:  eta: 0:01:36  iter: 1399  total_loss: 0.7129  loss_cls: 0.1349  loss_box_reg: 0.2912  loss_mask: 0.1398  loss_rpn_cls: 0.009904  loss_rpn_loc: 0.1365  time: 0.9557  data_time: 0.3115  lr: 5e-05  max_mem: 0M
[03/05 04:45:48 d2.utils.events]:  